In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import datetime
from datetime import datetime
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix, confusion_matrix, precision_score, recall_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df.isna().sum()

encounter_id                       0
patient_id                         0
hospital_id                        0
age                             4228
bmi                             3429
                               ...  
solid_tumor_with_metastasis      715
apache_3j_bodysystem            1662
apache_2_bodysystem             1662
Unnamed: 83                    91713
hospital_death                     0
Length: 85, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Data columns (total 85 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   encounter_id                   91713 non-null  int64  
 1   patient_id                     91713 non-null  int64  
 2   hospital_id                    91713 non-null  int64  
 3   age                            87485 non-null  float64
 4   bmi                            88284 non-null  float64
 5   elective_surgery               91713 non-null  int64  
 6   ethnicity                      90318 non-null  object 
 7   gender                         91688 non-null  object 
 8   height                         90379 non-null  float64
 9   icu_admit_source               91601 non-null  object 
 10  icu_id                         91713 non-null  int64  
 11  icu_stay_type                  91713 non-null  object 
 12  icu_type                       91713 non-null 

In [5]:
df.drop(["encounter_id" ,"patient_id" ,"hospital_id","Unnamed: 83", ],axis =1, inplace=True) # Not relevant to predict if a patient died or not

In [6]:
# Dealing with numerical columns
Num=[]
for col in df.columns:
    if (df[col].dtype==int)or (df[col].dtype==float):
        Num.append(col)


for col in df.columns:
    if col in Num :
        df[col].fillna(df[col].median(), inplace=True)
        
        
# Dealing with categorical columns
categorical=[]
for col in df.columns:
    if  (df[col].dtype==object):
        categorical.append(col)

for col in df.columns:
    if col in categorical:
        df[col].fillna(df[col].mode(), inplace=True)

In [7]:
!pip install category_encoders

In [8]:
from category_encoders import CountEncoder
CE = CountEncoder(normalize=True, cols=categorical)
data = CE.fit_transform(df)

In [9]:
from sklearn.preprocessing import RobustScaler
RS = RobustScaler()
scale = RS.fit_transform(data)
data = pd.DataFrame(scale, columns=data.columns)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

In [17]:
# Random forest implementation
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96     16756
         1.0       0.71      0.27      0.39      1587

    accuracy                           0.93     18343
   macro avg       0.82      0.63      0.68     18343
weighted avg       0.92      0.93      0.91     18343



In [14]:
from sklearn.model_selection import RandomizedSearchCV
import pprint 
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 30, num = 5)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pp = pprint.PrettyPrinter(width=41, compact=True)
pp.pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 15, 20, 25, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 800, 1400, 2000]}


In [30]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 800, 1400, 2000]},
                   random_state=42, verbose=2)

In [32]:
pred = rf_random.predict(X_test)
print(rf_random.best_estimator_)
print(classification_report(y_test, pred))


In [12]:
# Running LDA
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 800, 1400, 2000]},
                   random_state=42, verbose=2)

In [16]:
pred = rf_random.predict(X_test)
print(rf_random.best_estimator_)
print(classification_report(y_test, pred))

RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=1400)
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96     16756
         1.0       0.62      0.22      0.33      1587

    accuracy                           0.92     18343
   macro avg       0.78      0.61      0.64     18343
weighted avg       0.90      0.92      0.90     18343



In [15]:
# Running PCA
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['hospital_death'],axis = 1), data['hospital_death'], test_size=0.2, random_state=42)

from sklearn.decomposition import PCA
pca = PCA(n_components=0.7) # I tried 0.5 and 0.8, worser results
X_train = pca.fit_transform(X_train, y_train)
X_test = pca.transform(X_test)

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 800, 1400, 2000]},
                   random_state=42, verbose=2)

In [17]:
pred = rf_random.predict(X_test)
print(rf_random.best_estimator_)
print(classification_report(y_test, pred))

RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=1400)
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96     16756
         1.0       0.67      0.25      0.36      1587

    accuracy                           0.92     18343
   macro avg       0.80      0.62      0.66     18343
weighted avg       0.91      0.92      0.91     18343

